In [1]:
"""
Debug script to test Perplexity API responses and identify issues.
"""

import os
import json
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
api_key = os.environ.get("PERPLEXITY_API_KEY")

if not api_key:
    raise ValueError("PERPLEXITY_API_KEY environment variable not set")

# API endpoint
base_url = "https://api.perplexity.ai/chat/completions"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json",
}

# Test parameters
company_name = "granica.ai"
company_url = "https://granica.ai/"
duration = "1 Month"

# Simple system and user prompts
system_prompt = "You are a helpful assistant that finds URLs related to companies."
user_prompt = f"Find URLs related to the company '{company_name}' (their website is {company_url}). Focus on recent information from the past {duration}."

print("=== TESTING BASIC API CONNECTIVITY ===")
# Test 1: Simple request without structured output
simple_payload = {
    "model": "sonar-pro",
    "messages": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    "temperature": 0.2,
    "max_tokens": 1000,
}

try:
    print("Sending simple request...")
    response = requests.post(base_url, headers=headers, json=simple_payload)
    response.raise_for_status()
    print(f"Status Code: {response.status_code}")
    print(f"Response Type: {type(response.json())}")
    print("First 500 chars of response text:")
    print(json.dumps(response.json(), indent=2)[:500] + "...")
except Exception as e:
    print(f"Error with simple request: {str(e)}")

print("\n=== TESTING STRUCTURED OUTPUT ===")
# Test 2: Request with JSON schema
schema_payload = {
    "model": "sonar-pro",
    "messages": [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": user_prompt + " Return results in the specified JSON format.",
        },
    ],
    "temperature": 0.2,
    "max_tokens": 4000,
    "response_format": {
        "type": "json_schema",
        "json_schema": {
            "schema": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "url": {"type": "string"},
                        "title": {"type": "string"},
                        "description": {"type": "string"},
                    },
                    "required": ["url", "title", "description"],
                },
            }
        },
    },
}

try:
    print("Sending request with JSON schema...")
    response = requests.post(base_url, headers=headers, json=schema_payload)
    response.raise_for_status()
    json_response = response.json()
    print(f"Status Code: {response.status_code}")

    # Inspect the response structure
    print("Response keys:", json_response.keys())
    if "choices" in json_response and json_response["choices"]:
        choice = json_response["choices"][0]
        print("Choice keys:", choice.keys())

        if "message" in choice:
            message = choice["message"]
            print("Message keys:", message.keys())

            if "content" in message:
                content = message["content"]
                print(f"Content type: {type(content)}")
                # Try to parse JSON if it's a string
                if isinstance(content, str):
                    try:
                        parsed_content = json.loads(content)
                        print(
                            "Parsed content (first item):",
                            parsed_content[0] if parsed_content else "Empty",
                        )
                    except json.JSONDecodeError:
                        print(
                            "Content is not valid JSON. First 100 chars:", content[:100]
                        )
                else:
                    print("Content preview:", content)
    else:
        print("No choices in response")
        print("Full response:", json_response)

except Exception as e:
    print(f"Error with JSON schema request: {str(e)}")

=== TESTING BASIC API CONNECTIVITY ===
Sending simple request...
Status Code: 200
Response Type: <class 'dict'>
First 500 chars of response text:
{
  "id": "8dc64628-8af0-4d33-8802-a2fa7faffb4c",
  "model": "sonar-pro",
  "created": 1743474312,
  "usage": {
    "prompt_tokens": 45,
    "completion_tokens": 150,
    "total_tokens": 195,
    "citation_tokens": 12000,
    "num_search_queries": 1
  },
  "citations": [
    "https://bastakiss.com/blog/news-2/latest-developments-in-artificial-intelligence-march-2025-update-759",
    "https://granica.ai/ai_data_platform/",
    "https://www.neudesic.com/blog/top-5-ai-trends-march-2025/",
    "http...

=== TESTING STRUCTURED OUTPUT ===
Sending request with JSON schema...
Status Code: 200
Response keys: dict_keys(['id', 'model', 'created', 'usage', 'citations', 'object', 'choices'])
Choice keys: dict_keys(['index', 'finish_reason', 'message', 'delta'])
Message keys: dict_keys(['role', 'content'])
Content type: <class 'str'>
Parsed content (first 